## Neural Networks

- Can be constructed using the `torch.nn` package
  
- `nn` depends on `autograd` to define and differentiate models
  
- an `nn.Module` contains layers and a method `forward(input)` that returns the output

The tpical training procedure for a neural network is as follows:

- Define the neural network that has some learnable parameters (or weights)

- Iterate over a dataset of inputs

- Process input through the network

- Compute the loss (how far is the output from being correct)

- Propagate gradients back into the network’s parameters

- Update the weights of the network, typically using a simple update rule: `weight = weight - learning_rate * gradient`

##### Define the network

Note that we only define the `forward` function and the `backward` function is automatically defined for you using `autograd`

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


Learnable parameters are returned by `net.parameters()`

In [7]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1's weight

10
torch.Size([6, 1, 5, 5])


Let's try a random 32x32 input

In [9]:
input = torch.randn(1,1,32,32)
out = net(input)
print(out)

tensor([[-0.0569,  0.1090,  0.1010,  0.1427,  0.0146, -0.0622,  0.0592, -0.0616,
          0.0943, -0.1169]], grad_fn=<AddmmBackward0>)


Zero the gradient buffers of all parameters and backprops with random gradients

In [10]:
net.zero_grad()
out.backward(torch.randn(1,10))

### Loss Function

A Loss function takes the (output, target) pair of inputs, and computs a value that estimates how far away the output is from the target.
  
There are several different loss functions under the `nn` package. A simple loss is `nn.MSELoss` which omputes the mean-squared error between the output and the target

In [12]:
output = net(input)
target = torch.randn(10) # a dummy target, for example
target = target.view(1,-1) # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.1239, grad_fn=<MseLossBackward0>)


If you follow `loss` in the backward direction using the `.grad_fn` attribute, you will see a graph of computations that look like this:

```
input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> flatten -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss
```

When we call `loss.backward()`, the whole graph is differentiated w.r.t. the neural net parameters, and all Tensors in the graph that have `requires_grad=True` will have their `.grad` Tensor accumulated with the gradient

In [17]:
print(loss.grad_fn) # MSELoss
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLU

### Backprop

To backpop the error, we need to call `loss.backward()`. You need to clear existing gradients first though, or else gradient will accumulated to existing gradients.

Now we shall call `loss.backward()` and have a look at conv1's bias gradient before and after back propagation

In [18]:
net.zero_grad() # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backprop')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backprop')
print(net.conv1.bias.grad)

conv1.bias.grad before backprop
None
conv1.bias.grad after backprop
tensor([ 0.0072,  0.0017,  0.0146,  0.0073, -0.0040,  0.0030])


### Update the weights

The simplest update used in practice is the Stochastic Gradient Descent (SGD):

```
weight = weight - learning_rate * gradient
```

In [19]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

However, `torch.optim` allows you to use various different update rules

In [20]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad() # zero out gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() # executes the update